In [ ]:
# Get the data from the project
# Web scraping from https://proxyway.com/guides/web-scraping-with-python#:~:text=Steps%20to%20Build%20a%20Python%20Web%20Scraper%201,parameters.%20...%203%20Step%203%3A%20Write%20the%20Script
url = "https://www.irishgrassland.ie/journals/"
r = requests.get(url)
soup = BeautifulSoup(r.content)
soup = soup.find('table')
soup = soup.find_all('a')

In [ ]:
# download the pdfs from Irish Grassland Journal and save them in the Raw folder
# parse the url for the file name

for element in soup:
    url = element.get('href')
    end = url.rfind('_')
    if end > 0 : # set this condition as some of the links do not have the underscore in the file name. And these pdfs do not seem to be relevant to the project
        start = end - 4
        destFilename = 'Irish Grassland and Animal Production Association Journal' + url[start:end]
        rq.urlretrieve(url, './Data/Raw/' + destFilename + '.pdf')  


In [ ]:
# extract the text from the pdfs and save them raw folder
# reference https://www.geeksforgeeks.org/working-with-pdf-files-in-python/

files = os.listdir('./Data/Raw/')
files = filter(lambda f: f.endswith(('.pdf','.PDF')), files)

for f in files:
    pdfFileObj = open('./Data/Raw/' + f, 'rb')
    pdfReader = PyPDF2.PdfReader(pdfFileObj)
    num_pages = len(pdfReader.pages)
    #pageobj = pdfReader.pages[num_pages + 1]
    count = 0
    text = ""
    #text = pageobj.extractText()
    txtFilename = './Data/Raw/' + f.replace('.pdf', '.txt')
    
    while count < num_pages:
        pageObj = pdfReader.pages[count]
        count +=1
        text += pageObj.extract_text()
   
    txtFile = open(txtFilename, 'a', encoding="utf-8")
    txtFile.writelines(text) 

In [ ]:
# download the various articles that show in this search https://www.ifa.ie/?s=silage
# There are 8 pages of results for this search, so I have to loop through the pages and extract the links pages are structured as https://www.ifa.ie/page/2/?s=silage

# header code re engineered from https://stackoverflow.com/questions/41946166/requests-get-returns-403-while-the-same-url-works-in-browser
# only want to declare once in the code
h = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like

# intialise search and load first page to get the number of pages
soup = searchIFA(searchTerm = 'silage', pageNumber = 1, header = h)

# find the number of pages returned by the search
nav_results = soup.find_all("div", {"class": "nav-links"})
nav_results = nav_results[0].find_all('a')

for element in nav_results:
    if element.getText() == 'Next':
        break
    max_page = element.getText()
    max_page = int(max_page)

# loop through the pages and extract the links
for page in range(1, max_page + 1):
    print(page)
    soup = searchIFA(searchTerm = 'silage', pageNumber = page, header = h)
    searchLinks = load_links(soup = soup)
    # now that I have the url, I can use beautiful soup to extract the text from the article
    for item in searchLinks:
        resultText(url = searchLinks[item], destFilename = item, header=h)
   